# PySpark Tutorial
Deciding whether or not Spark is the best solution for your problem takes some experience, but you can consider questions like:

1. Is my data too big to work with on a single machine?
2. Can my calculations be easily parallelized?

PySpark is a great language for performing exploratory data analysis at scale, building machine learning pipelines, and creating ETLs for a data platform. If you’re already familiar with Python and libraries such as Pandas, then PySpark is a great language to learn in order to create more scalable analyses and pipelines. The goal of this post is to show how to get up and running with PySpark and to perform common tasks.

The first step in using Spark is connecting to a cluster.

In practice, the cluster will be hosted on a remote machine that's connected to all other nodes. There will be one computer, called the master that manages splitting up the data and the computations. The master is connected to the rest of the computers in the cluster, which are called slaves. The master sends the slaves data and calculations to run, and they send their results back to the master.


In [1]:
from pyspark import SparkConf, HiveContext, SparkContext
from pyspark.sql import SparkSession, Row, SQLContext
from pyspark.sql import functions as F

In [2]:
# Set Spark Context

conf = SparkConf()
conf.set('spark.executor.memory','10g')\
    .set('spark.yarn.queue', 'root.odfgsbx2_q1')\
    .set('spark.executor.cores','5')\
    .set('spark.executor.instances','3')\
    
spark = SparkSession.builder.appName('DSU') \
    .config(conf=conf) \
    .getOrCreate()
    
# Default Spark context
#sc, sqlContext = setupSparkContext()

In [3]:
sqlContext = SQLContext(spark.sparkContext)

### Understanding the parameters for the spark context: 

spark.executor.memory: heap size or memory for each executor <br /> <br />
spark.yarn.queue : sending job to specified YARN Queue<br /> <br />
spark.executor.cores : specify number of cores (controls number of concurrent tasks an executor can run), for the context above each executor can run a maximum of 5 tasks at the same time <br /> <br />
spark.executor.instances : number of executors requested <br />

https://blog.cloudera.com/blog/2015/03/how-to-tune-your-apache-spark-jobs-part-2/ <br />
https://spark.apache.org/docs/latest/cluster-overview.html 

## (Py)Spark's core data structure and DataFrames

Spark's core data structure is the Resilient Distributed Dataset (RDD). This is a low level object that lets Spark work its magic by splitting data across multiple nodes in the cluster. However, RDDs are extremely hard to work with directly and so we  will use DataFrames.

We learned about Data Frames towards the start of this course. These Spark DFs are really similar to pandas DF, except that they are on steroids and a few syntax is different. Spark DFs are optimized to work with RDD, so internally all the transformations and manipulations occur in Spark RDD.

Spark dataframe can be thought of as a table distributed across a cluster and has functionality that is similar to dataframes in R and Pandas. If you want to do distributed computation using PySpark, then you’ll need to perform operations on Spark dataframes, and not other python data types.


In [7]:
spark.catalog.listTables()

[]

Most of the time, you will be reading in an external file. There are 2 ways of reading in a file:

In [8]:
import pandas as pd
df = pd.read_csv('Mall_Customers.csv')
type(df)
df.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


![title](https://s3.amazonaws.com/assets.datacamp.com/production/course_4452/datasets/spark_figure.png)

The `.createDataFrame()` method takes a pandas DF and returns a Spark DF.
The output of this method is stored locally, not in the SparkSession catalog. This means that you can use all the Spark DataFrame methods on it, but you can't access the data in other contexts (unless you upload it).



There is a method to store things in SparkSession catalog in temp space.
You can do this using the `.createTempView()` Spark DataFrame method, which takes as its only argument the name of the temporary table you'd like to register. This method registers the DataFrame as a table in the catalog, but as this table is temporary, it can only be accessed from the specific SparkSession used to create the Spark DataFrame. (Similar to sql temp tables)

There is also the method `.createOrReplaceTempView()`. This safely creates a new temporary table if nothing was there before, or updates an existing table if one was already defined. You'll use this method to avoid running into problems with duplicate tables.

In [9]:
## read it in as a pandas and convert it to a pyspark dataframe
spk_df = spark.createDataFrame(df)
spk_df.show(5)

+----------+------+---+------------------+----------------------+
|CustomerID|Gender|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
+----------+------+---+------------------+----------------------+
only showing top 5 rows



### Easily convert Spark DFs to pandas
####  A word of caution, pandas can handle 10 mil rows, but the constraint is RAM. We don't want you to be the person blamed to bring down the BDPaaS queue :)
This function should generally be avoided except when working with small dataframes, because it pulls the entire object into memory on a single node.


https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html


In [11]:
pd_df = spk_df.toPandas()
pd_df.head(5)

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


##### One of the key differences between Pandas and Spark dataframes is eager versus lazy execution. In PySpark, operations are delayed until a result is actually needed in the pipeline.

For example, you can specify operations for loading a data set from S3 and applying a number of transformations to the dataframe, but these operations won’t immediately be applied. Instead, a graph of transformations is recorded, and once the data is actually needed, for example when writing the results back to S3, then the transformations are applied as a single pipeline operation. This approach is used to avoid pulling the full data frame into memory and enables more effective processing across a cluster of machines. With Pandas dataframes, everything is pulled into memory, and every Pandas operation is immediately applied.

### Reading a file (CSV, Parquets etc) directly into spark

In [12]:
mylist = [
  {"name":'Alice',"age":"10"},
  {"name":'Owen',"age":"11"},
  {"name":'Ryan',"age":"12"}
]

local_df = spark.createDataFrame(Row(**x) for x in mylist)

In [13]:
local_df.show()
local_df.count()

+---+-----+
|age| name|
+---+-----+
| 10|Alice|
| 11| Owen|
| 12| Ryan|
+---+-----+



3

### read it in as a csv directly

In [22]:

mall_customers_df = spark.read.csv("maprfs:///datalake/optum/optuminsight/sandbox3/dsu/nilay_bhatt/Mall_Customers.csv", header=True)

In [41]:
mall_customers_df.show(5)
mall_customers_df.printSchema()

+----------+------+---+------------------+----------------------+
|CustomerID|Gender|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
+----------+------+---+------------------+----------------------+
only showing top 5 rows

root
 |-- CustomerID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Annual Income (k$): string (nullable = true)
 |-- Spending Score (1-100): string (nullable = true)



## Data Manipulation: let the fun began!!!! 
#### A lot of things are similar from pandas but some nuances

Things such as .columns, .describe, .head, are similar. We now have withColumns, UDFs, SparkSQL that we can work with

In [37]:
mall_customers_df.columns

['CustomerID', 'Gender', 'Age', 'Annual Income (k$)', 'Spending Score (1-100)']

### Filtering Data
#### You can select columns either way, but the first way is slightly more common
A large part of working with DataFrames is the ability to quickly filter out data based on conditions. Spark DataFrames are built on top of the Spark SQL platform, which means that is you already know SQL, you can quickly and easily grab that data using SQL commands, or using the DataFrame methods

In [42]:
mall_customers_df.select(F.col('Gender'), F.col('Age')).show()
#df_trim_1 = mall_customers_df.select(df['Gender'], df['Age'])

+------+---+
|Gender|Age|
+------+---+
|  Male| 19|
|  Male| 21|
|Female| 20|
|Female| 23|
|Female| 31|
|Female| 22|
|Female| 35|
|Female| 23|
|  Male| 64|
|Female| 30|
|  Male| 67|
|Female| 35|
|Female| 58|
|Female| 24|
|  Male| 37|
|  Male| 22|
|Female| 35|
|  Male| 20|
|  Male| 52|
|Female| 35|
+------+---+
only showing top 20 rows



In [48]:
mall_customers_df.filter(F.col('Age')>18).show()
#mall_customers_df.filter('Age>18').show()

+----------+------+---+------------------+----------------------+
|CustomerID|Gender|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
|         6|Female| 22|                17|                    76|
|         7|Female| 35|                18|                     6|
|         8|Female| 23|                18|                    94|
|         9|  Male| 64|                19|                     3|
|        10|Female| 30|                19|                    72|
|        11|  Male| 67|                19|                    14|
|        12|Female| 35|                19|                    99|
|        1

You can also chain functions. This is actually recomended way of working. Although as we are starting out, we might want to right out seperate commands, Spark also optimizes our code.

![title](DAG.png)

In [50]:
mall_customers_df.filter(F.col('Age')>18).select(F.col('Gender'), F.col('Annual Income (k$)')).show()

+------+------------------+
|Gender|Annual Income (k$)|
+------+------------------+
|  Male|                15|
|  Male|                15|
|Female|                16|
|Female|                16|
|Female|                17|
|Female|                17|
|Female|                18|
|Female|                18|
|  Male|                19|
|Female|                19|
|  Male|                19|
|Female|                19|
|Female|                20|
|Female|                20|
|  Male|                20|
|  Male|                20|
|Female|                21|
|  Male|                21|
|  Male|                23|
|Female|                23|
+------+------------------+
only showing top 20 rows



In [51]:
mall_customers_df.filter(F.col('Age')>18).select(['Gender','Annual Income (k$)']).show()

+------+------------------+
|Gender|Annual Income (k$)|
+------+------------------+
|  Male|                15|
|  Male|                15|
|Female|                16|
|Female|                16|
|Female|                17|
|Female|                17|
|Female|                18|
|Female|                18|
|  Male|                19|
|Female|                19|
|  Male|                19|
|Female|                19|
|Female|                20|
|Female|                20|
|  Male|                20|
|  Male|                20|
|Female|                21|
|  Male|                21|
|  Male|                23|
|Female|                23|
+------+------------------+
only showing top 20 rows



Using normal python comparison operators is another way to do this, they will look very similar to SQL operators, except you need to make sure you are calling the entire column within the dataframe, using the format: df["column name"]

In [53]:
mall_customers_df.filter(mall_customers_df["Gender"] == 'Male').show()

+----------+------+---+------------------+----------------------+
|CustomerID|Gender|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         9|  Male| 64|                19|                     3|
|        11|  Male| 67|                19|                    14|
|        15|  Male| 37|                20|                    13|
|        16|  Male| 22|                20|                    79|
|        18|  Male| 20|                21|                    66|
|        19|  Male| 52|                23|                    29|
|        21|  Male| 35|                24|                    35|
|        22|  Male| 25|                24|                    73|
|        24|  Male| 31|                25|                    73|
|        26|  Male| 29|                28|                    82|
|        2

In [55]:
mall_customers_df.filter((mall_customers_df["Gender"] == "Male") & (mall_customers_df["Spending Score (1-100)"] < 50)).show()

+----------+------+---+------------------+----------------------+
|CustomerID|Gender|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         9|  Male| 64|                19|                     3|
|        11|  Male| 67|                19|                    14|
|        15|  Male| 37|                20|                    13|
|        19|  Male| 52|                23|                    29|
|        21|  Male| 35|                24|                    35|
|        31|  Male| 60|                30|                     4|
|        33|  Male| 53|                33|                     4|
|        43|  Male| 48|                39|                    36|
|        56|  Male| 47|                43|                    41|
|        58|  Male| 69|                44|                    46|
|        60|  Male| 53|                46|                    46|
|        7

In [59]:
mall_customers_df.filter((mall_customers_df["Gender"] == "Male") & (mall_customers_df["Spending Score (1-100)"] < 5)).collect()

[Row(CustomerID='9', Gender='Male', Age='64', Annual Income (k$)='19', Spending Score (1-100)='3'),
 Row(CustomerID='31', Gender='Male', Age='60', Annual Income (k$)='30', Spending Score (1-100)='4'),
 Row(CustomerID='33', Gender='Male', Age='53', Annual Income (k$)='33', Spending Score (1-100)='4'),
 Row(CustomerID='157', Gender='Male', Age='37', Annual Income (k$)='78', Spending Score (1-100)='1'),
 Row(CustomerID='159', Gender='Male', Age='34', Annual Income (k$)='78', Spending Score (1-100)='1')]

### Rows can be called to turn into dictionaries

We can also grab data by column or as a single column DF using df['Age'] or df.select('Age'). The latter gives a DF

### The function ``.withColumn`` will create a new column if the name of the new column is different from the original columns. If the name of the new column is the same to an original column name, then that column will be replaced with the new values. 

In [110]:
df = mall_customers_df.withColumn('Annual Income (k$)', F.col('Annual Income (k$)') * 1000)
df = df.withColumnRenamed('Annual Income (k$)', 'Annual Income') ## Rename Annual Income (k$) to Annual Income

In [62]:
df.show()

+----------+------+---+-------------+----------------------+
|CustomerID|Gender|Age|Annual Income|Spending Score (1-100)|
+----------+------+---+-------------+----------------------+
|         1|  Male| 19|      15000.0|                    39|
|         2|  Male| 21|      15000.0|                    81|
|         3|Female| 20|      16000.0|                     6|
|         4|Female| 23|      16000.0|                    77|
|         5|Female| 31|      17000.0|                    40|
|         6|Female| 22|      17000.0|                    76|
|         7|Female| 35|      18000.0|                     6|
|         8|Female| 23|      18000.0|                    94|
|         9|  Male| 64|      19000.0|                     3|
|        10|Female| 30|      19000.0|                    72|
|        11|  Male| 67|      19000.0|                    14|
|        12|Female| 35|      19000.0|                    99|
|        13|Female| 58|      20000.0|                    15|
|        14|Female| 24| 

Next, let us create a new feature: annual income 

In [111]:
df = df.withColumn('Annual_Income/Spending_Score', F.col('Annual Income')/F.col('Spending Score (1-100)'))

In [64]:
df.show()

+----------+------+---+-------------+----------------------+----------------------------+
|CustomerID|Gender|Age|Annual Income|Spending Score (1-100)|Annual_Income/Spending_Score|
+----------+------+---+-------------+----------------------+----------------------------+
|         1|  Male| 19|      15000.0|                    39|          384.61538461538464|
|         2|  Male| 21|      15000.0|                    81|           185.1851851851852|
|         3|Female| 20|      16000.0|                     6|          2666.6666666666665|
|         4|Female| 23|      16000.0|                    77|           207.7922077922078|
|         5|Female| 31|      17000.0|                    40|                       425.0|
|         6|Female| 22|      17000.0|                    76|          223.68421052631578|
|         7|Female| 35|      18000.0|                     6|                      3000.0|
|         8|Female| 23|      18000.0|                    94|          191.48936170212767|
|         

### Let us convert gender to binary / Normalizing the data

### F.when() functions like an if/else statement. For example, if the gender is male then put the value equal to 0, else put the value to 1 

In [112]:
df = df.withColumn('Gender_Binary', F.when(F.col('Gender') == 'Male', 0).otherwise(1))

In [66]:
df.show()

+----------+------+---+-------------+----------------------+----------------------------+-------------+
|CustomerID|Gender|Age|Annual Income|Spending Score (1-100)|Annual_Income/Spending_Score|Gender_Binary|
+----------+------+---+-------------+----------------------+----------------------------+-------------+
|         1|  Male| 19|      15000.0|                    39|          384.61538461538464|            0|
|         2|  Male| 21|      15000.0|                    81|           185.1851851851852|            0|
|         3|Female| 20|      16000.0|                     6|          2666.6666666666665|            1|
|         4|Female| 23|      16000.0|                    77|           207.7922077922078|            1|
|         5|Female| 31|      17000.0|                    40|                       425.0|            1|
|         6|Female| 22|      17000.0|                    76|          223.68421052631578|            1|
|         7|Female| 35|      18000.0|                     6|    

You can describe a dataframe similar to R to extract key statistics. 

In [113]:
df.drop('Gender').drop('Annual_Income/Spending_Score').describe().show()

+-------+------------------+-----------------+------------------+----------------------+------------------+
|summary|        CustomerID|              Age|     Annual Income|Spending Score (1-100)|     Gender_Binary|
+-------+------------------+-----------------+------------------+----------------------+------------------+
|  count|               200|              200|               200|                   200|               200|
|   mean|             100.5|            38.85|           60560.0|                  50.2|              0.56|
| stddev|57.879184513951124|13.96900733155888|26264.721165271247|    25.823521668370173|0.4976325863071808|
|    min|                 1|               18|           15000.0|                     1|                 0|
|    max|                99|               70|          137000.0|                    99|                 1|
+-------+------------------+-----------------+------------------+----------------------+------------------+



### UDFs
The following example is a UDF (user defined function). UDFs are means to apply Python functions to PySpark dataframes 
row by row. Before applying an UDF, make sure there are no pre-defined functions in PySpark already. UDFs are
computationally expensive. 

In [114]:
def square(x): 
    return int(x)**2

square_udf = F.udf(lambda z: square(z))

df = df.withColumn('udf_test', square_udf(F.col('Spending Score (1-100)')))

In [99]:
df.show()

+----------+------+---+-------------+----------------------+----------------------------+-------------+--------+
|CustomerID|Gender|Age|Annual Income|Spending Score (1-100)|Annual_Income/Spending_Score|Gender_Binary|udf_test|
+----------+------+---+-------------+----------------------+----------------------------+-------------+--------+
|         1|  Male| 19|      15000.0|                    39|          384.61538461538464|            0|    1521|
|         2|  Male| 21|      15000.0|                    81|           185.1851851851852|            0|    6561|
|         3|Female| 20|      16000.0|                     6|          2666.6666666666665|            1|      36|
|         4|Female| 23|      16000.0|                    77|           207.7922077922078|            1|    5929|
|         5|Female| 31|      17000.0|                    40|                       425.0|            1|    1600|
|         6|Female| 22|      17000.0|                    76|          223.68421052631578|       

In [115]:
df = df.withColumnRenamed('Annual Income', 'Annual_Income')
df = df.withColumnRenamed('udf_test', 'spend_square')
df.show()

+----------+------+---+-------------+----------------------+----------------------------+-------------+------------+
|CustomerID|Gender|Age|Annual_Income|Spending Score (1-100)|Annual_Income/Spending_Score|Gender_Binary|spend_square|
+----------+------+---+-------------+----------------------+----------------------------+-------------+------------+
|         1|  Male| 19|      15000.0|                    39|          384.61538461538464|            0|        1521|
|         2|  Male| 21|      15000.0|                    81|           185.1851851851852|            0|        6561|
|         3|Female| 20|      16000.0|                     6|          2666.6666666666665|            1|          36|
|         4|Female| 23|      16000.0|                    77|           207.7922077922078|            1|        5929|
|         5|Female| 31|      17000.0|                    40|                       425.0|            1|        1600|
|         6|Female| 22|      17000.0|                    76|    

## Spark SQL

Spark SQL is a new module in Spark which integrates relational processing with Spark’s functional programming API. It supports querying data either via SQL or via the Hive Query Language.

For those of you familiar with RDBMS, Spark SQL will be an easy transition from your earlier tools where you can extend the boundaries of traditional relational data processing.

Spark SQL integrates relational processing with Spark’s functional programming. It provides support for various data sources and makes it possible to weave SQL queries with code transformations thus resulting in a very powerful tool.

![title](https://d1jnx9ba8s6j9r.cloudfront.net/blog/wp-content/uploads/2016/12/Spark-SQL-Architecture-Spark-SQL-Edureka-1.png)

https://www.edureka.co/blog/spark-sql-tutorial/

### You can query the data through sql by the .sql function. Remember, if you are trying to call queries locally and not through a database, then you have to create a temp view (createOrReplaceTempView). 

In [116]:
df.createOrReplaceTempView("table1")
sql_df = sqlContext.sql("SELECT Age FROM table1 WHERE Age < 20")
sql_df.show()

+---+
|Age|
+---+
| 19|
| 18|
| 19|
| 18|
| 19|
| 18|
| 19|
| 19|
| 18|
| 19|
| 19|
| 19|
+---+



In [125]:
spark.sql("SELECT * FROM table1 WHERE Annual_Income < 18000").show()

+----------+------+---+-------------+----------------------+----------------------------+-------------+------------+
|CustomerID|Gender|Age|Annual_Income|Spending Score (1-100)|Annual_Income/Spending_Score|Gender_Binary|spend_square|
+----------+------+---+-------------+----------------------+----------------------------+-------------+------------+
|         1|  Male| 19|      15000.0|                    39|          384.61538461538464|            0|        1521|
|         2|  Male| 21|      15000.0|                    81|           185.1851851851852|            0|        6561|
|         3|Female| 20|      16000.0|                     6|          2666.6666666666665|            1|          36|
|         4|Female| 23|      16000.0|                    77|           207.7922077922078|            1|        5929|
|         5|Female| 31|      17000.0|                    40|                       425.0|            1|        1600|
|         6|Female| 22|      17000.0|                    76|    

### Convert the dataframe to a bunch of lists

In [141]:
lists = df.collect()
lists[0]

Row(CustomerID='1', Gender='Male', Age='19', Annual_Income=15000.0, Spending Score (1-100)='39', Annual_Income/Spending_Score=384.61538461538464, Gender_Binary=0, spend_square='1521')

## Groupby spending score and sum all the columns

Let's learn how to use GroupBy and Aggregate methods on a DataFrame. GroupBy allows you to group rows together based off some column value, for example, you could group together sales data by the day the sale occured, or group repeast customer data based off the name of the customer. Once you've performed the GroupBy operation you can use an aggregate function off that data. An aggregate function aggregates multiple rows of data into a single output, such as taking the sum of inputs, or counting the number of inputs.

In [142]:
df.groupby(F.col('Spending Score (1-100)')).sum().show() ## ignores columns that are type string

+----------------------+------------------+---------------------------------+------------------+
|Spending Score (1-100)|sum(Annual_Income)|sum(Annual_Income/Spending_Score)|sum(Gender_Binary)|
+----------------------+------------------+---------------------------------+------------------+
|                    51|          148000.0|               2901.9607843137255|                 1|
|                     7|           73000.0|                10428.57142857143|                 1|
|                    15|          206000.0|               13733.333333333332|                 1|
|                    54|          160000.0|                2962.962962962963|                 2|
|                    11|           71000.0|                6454.545454545455|                 0|
|                    29|           93000.0|               3206.8965517241377|                 1|
|                    69|          191000.0|               2768.1159420289855|                 1|
|                    42|      

### You should groupby Spending Score and aggergate Age by the count as well as averaging the Annual Income. 



In [144]:
df.groupby(F.col('Spending Score (1-100)')).agg({"Age":"count", "Annual_Income":"avg"}).show()

+----------------------+------------------+----------+
|Spending Score (1-100)|avg(Annual_Income)|count(Age)|
+----------------------+------------------+----------+
|                    51|49333.333333333336|         3|
|                     7|           73000.0|         1|
|                    15| 68666.66666666667|         3|
|                    54|53333.333333333336|         3|
|                    11|           71000.0|         1|
|                    29|           46500.0|         2|
|                    69|           95500.0|         2|
|                    42|           53750.0|         8|
|                    73|           44000.0|         6|
|                    87|           52500.0|         2|
|                     3|           19000.0|         1|
|                    34|           72000.0|         1|
|                    59|           54000.0|         5|
|                     8|          113000.0|         1|
|                    28|           82500.0|         2|
|         

## Counts, means, max, Min

In [150]:
df.groupBy("Gender").mean().show()
df.groupBy("Gender").count().show()
df.groupBy("Gender").max().show()
df.groupBy("Gender").min().show()

+------+------------------+---------------------------------+------------------+
|Gender|avg(Annual_Income)|avg(Annual_Income/Spending_Score)|avg(Gender_Binary)|
+------+------------------+---------------------------------+------------------+
|Female|           59250.0|               1713.7840423847515|               1.0|
|  Male| 62227.27272727273|                4237.268560481555|               0.0|
+------+------------------+---------------------------------+------------------+

+------+-----+
|Gender|count|
+------+-----+
|Female|  112|
|  Male|   88|
+------+-----+

+------+------------------+---------------------------------+------------------+
|Gender|max(Annual_Income)|max(Annual_Income/Spending_Score)|max(Gender_Binary)|
+------+------------------+---------------------------------+------------------+
|Female|          126000.0|                          15000.0|                 1|
|  Male|          137000.0|                          78000.0|                 0|
+------+---------

## OrderBy

In [151]:
df.orderBy("Annual_Income").show()
df.orderBy(df["Annual_Income"].desc()).show()

+----------+------+---+-------------+----------------------+----------------------------+-------------+------------+
|CustomerID|Gender|Age|Annual_Income|Spending Score (1-100)|Annual_Income/Spending_Score|Gender_Binary|spend_square|
+----------+------+---+-------------+----------------------+----------------------------+-------------+------------+
|         1|  Male| 19|      15000.0|                    39|          384.61538461538464|            0|        1521|
|         2|  Male| 21|      15000.0|                    81|           185.1851851851852|            0|        6561|
|         3|Female| 20|      16000.0|                     6|          2666.6666666666665|            1|          36|
|         4|Female| 23|      16000.0|                    77|           207.7922077922078|            1|        5929|
|         5|Female| 31|      17000.0|                    40|                       425.0|            1|        1600|
|         6|Female| 22|      17000.0|                    76|    

Check out this link for more info on other methods: http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module

Not all methods need a groupby call, instead you can just call the generalized .agg() method, that will call the aggregate across all rows in the dataframe column specified. It can take in arguments as a single column, or create multiple aggregate calls all at once using dictionary notation.

There are a variety of functions you can import from pyspark.sql.functions. Check out the documentation for the full list available: http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions

### Dates and Timestamps

In [133]:
from pyspark.sql.functions import format_number,dayofmonth,hour,dayofyear,month,year,weekofyear,date_format
date_df = spark.read.csv("maprfs:///datalake/optum/optuminsight/sandbox3/dsu/nilay_bhatt/appl_stock.csv",header=True,inferSchema=True)

In [134]:
date_df.show(5)

+-------------------+----------+----------+------------------+------------------+---------+------------------+
|               Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
o

In [137]:
date_df.select(dayofmonth(date_df['Date'])).show(5)

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
+----------------+
only showing top 5 rows



In [138]:
date_df.select(dayofyear(date_df['Date'])).show(5)

+---------------+
|dayofyear(Date)|
+---------------+
|              4|
|              5|
|              6|
|              7|
|              8|
+---------------+
only showing top 5 rows



In [140]:
date_df.withColumn("Year",year(date_df['Date'])).show(5)

+-------------------+----------+----------+------------------+------------------+---------+------------------+----+
|               Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|Year|
+-------------------+----------+----------+------------------+------------------+---------+------------------+----+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|2010|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|2010|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|2010|
|2010-01-07 00:00:00|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|2010|
|2010-01-08 00:00:00|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|2010|
+-------------------+----------+----------+------------------+----------